# Twelve Data Setup Guide

**Provider:** Twelve Data (Real-time & historical market data)

**Best for:** Technical indicators, forex, crypto, global stocks

**Time:** 5 minutes

**Cost:** FREE tier (800 credits/day, ~8 API calls/min), Pro from $29/mo

---

**Why Twelve Data?** Excellent technical analysis API with 100+ built-in indicators. Great for algorithmic trading strategies.

## Step 1: Create Twelve Data Account

1. Go to [twelvedata.com](https://twelvedata.com/)
2. Sign up

## Step 2: Get Your API Key

1. Go to [twelvedata.com/account/api-keys](https://twelvedata.com/account/api-keys)
2. Your API key is shown on the dashboard
3. **Copy the key**

## Step 3: Set Your API Key

Run this cell and paste your API key when prompted:

In [11]:
import os
from getpass import getpass

# Prompt for API key (hidden input)
api_key = getpass("Enter your Twelve Data API key: ")
os.environ["TWELVE_DATA_API_KEY"] = api_key

print("✅ API key set!")
print(f"   Key format: {api_key[:8]}...{api_key[-4:]}")

Enter your Twelve Data API key: ··········
✅ API key set!
   Key format: 0aabaf9e...bebf


## Step 4: Install Twelve Data Package

In [12]:
!pip install -q twelvedata pandas

## Step 5: Test Your Connection - Real-time Quote

In [13]:
from twelvedata import TDClient

# Initialize client
td = TDClient(apikey=os.environ["TWELVE_DATA_API_KEY"])

# Get real-time quote
quote = td.quote(symbol="AAPL").as_json()

print("✅ Connection successful!")
print(f"   Symbol: {quote.get('symbol')}")
print(f"   Name: {quote.get('name')}")
print(f"   Price: ${quote.get('close')}")
print(f"   Open: ${quote.get('open')}")
print(f"   High: ${quote.get('high')}")
print(f"   Low: ${quote.get('low')}")
print(f"   Volume: {quote.get('volume')}")

✅ Connection successful!
   Symbol: AAPL
   Name: Apple Inc.
   Price: $250.00000
   Open: $249.14000
   High: $250.75500
   Low: $248.39000
   Volume: 878588


## Step 6: Get Historical Data (Time Series)

In [14]:
# Get time series data
ts = td.time_series(
    symbol="AAPL",
    interval="1day",
    outputsize=30
)

df = ts.as_pandas()

print("✅ Historical data retrieved!")
print(f"   Period: {df.index[-1].date()} to {df.index[0].date()}")
print(f"   Records: {len(df)}")
print(f"\nLast 5 days:")
print(df.head())

✅ Historical data retrieved!
   Period: 2025-12-09 to 2026-01-22
   Records: 30

Last 5 days:
                 open       high        low      close    volume
datetime                                                        
2026-01-22  249.14000  250.75500  248.39000  250.00000    878588
2026-01-21  248.70000  251.56000  245.17999  247.64999  54446100
2026-01-20  252.73000  254.78999  243.42000  246.70000  80267500
2026-01-16  257.89999  258.89999  254.92999  255.53000  72142800
2026-01-15  260.64999  261.04001  257.04999  258.20999  39388600


## Step 7: Get Forex Data

In [15]:
# Get EUR/USD quote
forex = td.quote(symbol="EUR/USD").as_json()

print("✅ Forex Quote:")
print(f"   Pair: {forex.get('symbol')}")
print(f"   Rate: {forex.get('close')}")
print(f"   Open: {forex.get('open')}")
print(f"   High: {forex.get('high')}")
print(f"   Low: {forex.get('low')}")

✅ Forex Quote:
   Pair: EUR/USD
   Rate: 1.17250
   Open: 1.16911
   High: 1.17265
   Low: 1.16701


## Step 8: Create Tools for smolagents

In [16]:
!pip install -q smolagents

In [17]:
from smolagents import tool

@tool
def get_stock_quote_twelvedata(ticker: str) -> dict:
    """Get real-time stock quote from Twelve Data.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT')

    Returns:
        Dictionary with price, open, high, low, volume
    """
    from twelvedata import TDClient
    import os

    td = TDClient(apikey=os.environ["TWELVE_DATA_API_KEY"])
    quote = td.quote(symbol=ticker).as_json()

    return {
        "ticker": quote.get("symbol"),
        "name": quote.get("name"),
        "price": float(quote.get("close", 0)),
        "open": float(quote.get("open", 0)),
        "high": float(quote.get("high", 0)),
        "low": float(quote.get("low", 0)),
        "volume": int(quote.get("volume", 0)),
        "source": "Twelve Data"
    }

@tool
def get_historical_data_twelvedata(ticker: str, days: int = 30) -> dict:
    """Get historical stock data from Twelve Data.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT')
        days: Number of days of history (default 30)

    Returns:
        Dictionary with historical price data summary
    """
    from twelvedata import TDClient
    import os

    td = TDClient(apikey=os.environ["TWELVE_DATA_API_KEY"])
    ts = td.time_series(
        symbol=ticker,
        interval="1day",
        outputsize=days
    )
    df = ts.as_pandas()

    return {
        "ticker": ticker.upper(),
        "period_start": str(df.index[-1].date()),
        "period_end": str(df.index[0].date()),
        "records": len(df),
        "latest_close": float(df.iloc[0]['close']),
        "period_high": float(df['high'].max()),
        "period_low": float(df['low'].min()),
        "avg_volume": int(df['volume'].mean()),
        "source": "Twelve Data"
    }

# Test the tools
print("Tool 1 - Stock Quote:")
print(get_stock_quote_twelvedata("AAPL"))

print("\nTool 2 - Historical Data:")
print(get_historical_data_twelvedata("AAPL", 30))

Tool 1 - Stock Quote:
{'ticker': 'AAPL', 'name': 'Apple Inc.', 'price': 250.0, 'open': 249.14, 'high': 250.755, 'low': 248.39, 'volume': 878588, 'source': 'Twelve Data'}

Tool 2 - Historical Data:
{'ticker': 'AAPL', 'period_start': '2025-12-09', 'period_end': '2026-01-22', 'records': 30, 'latest_close': 250.0, 'period_high': 280.14999, 'period_low': 243.42, 'avg_volume': 43465576, 'source': 'Twelve Data'}


## ✅ Setup Complete!

You're ready to use Twelve Data in the course notebooks.

**Next steps:**
- Save your API key securely (password manager)
- Explore the 100+ technical indicators with `.with_{indicator}()`
- Use for algorithmic trading strategies
- Continue to the course Module 1 notebook
